# A*-search using `aima-python`

Algorithm implementations taken from [here](https://github.com/aimacode/aima-python/blob/master/search-4e.ipynb)

* *State* is defined by gifts in bags

* *Goal states* are defined by filled bags satisfying problem conditions

* *Actions* : put a gift in a bag with a minimal weight

In [1]:
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
np.random.seed(2016)

In [3]:
from search import Problem, astar_search, uniform_cost_search, state_sequence, action_sequence
import sys
sys.path.append('../common')
from utils import weight2 as weight_fn

In [4]:
n_bags = 50
max_weight = 50


n_horses = 1000
n_balls = 1100
n_bikes = 500
n_trains = 1000
n_coals = 166
n_books = 1200
n_dolls = 1000
n_blocks = 1000
n_gloves = 200

available_gifts = {
    "horse": n_horses,
    "ball": n_balls,
    "bike": n_bikes,
    "train": n_trains,
    "coal": n_coals,
    "book": n_books,
    "doll": n_dolls,
    "blocks": n_blocks,
    "gloves": n_gloves
}

type_cost = {
    "horse": 2,
    "ball": 3,
    "bike": 2,
    "train": 2,
    "coal": 1,
    "book": 5,
    "doll": 2,
    "blocks": 1,
    "gloves": 5    
}

gift_types = available_gifts.keys()
n_types = len(gift_types)

In [5]:
len(gift_types), gift_types

(9,
 dict_keys(['gloves', 'train', 'doll', 'coal', 'bike', 'horse', 'book', 'ball', 'blocks']))

State is tuple (bags) of tuples (gifts)

In [6]:
def bag_weight(bag, count=1):
    weight = []
    for c in range(count):
        w = 0
        for gift in bag:
            gift_type = gift.split('_')[0]
            w += weight_fn(gift_type, 100)
        weight.append(w)            
    return np.mean(weight)

In [7]:
class SantasBagsProblem(Problem):
            
    def actions(self, state):
        """Return a list of actions executable in this state."""                        
        # find a bag with a minimal weight                
        min_weight_bag_index = 0
        min_weight = self.max_weight
        for i, bag in enumerate(state):
            w = self.bag_weight_fn(bag)
            if min_weight > w:
                min_weight_bag_index = i
                min_weight = w
        
        actions = [(min_weight_bag_index, gift_type) for gift_type in self.gift_types]
#         print("actions: ", actions)
        return actions
    
    def result(self, state, action):
        """The state that results from executing this action in this state."""
        bag_id, gift_type  = action
        if self.available_gifts[gift_type] < 1:
#             print("No more gifts of type : ", gift_type)
            return state
        
#         print("-- result : input state: ", state, "action: ", action)
        bag_weight = self.bag_weight_fn(state[bag_id])
        
        gift_weight = self.weight_fn(gift_type, 100)
        if bag_weight + gift_weight > self.max_weight:
            return state
                
        new_state = list(state)
        gift = gift_type + '_%i' % self.available_gifts[gift_type]
        self.available_gifts[gift_type] -= 1
        bag = list(new_state[bag_id])
        bag.append(gift)
        new_state[bag_id] = tuple(bag)
        
#         print("-- result : output state: ", new_state)
        return tuple(new_state)

    def is_goal(self, state):
        """True if the state is a goal."""        
        mean_score = self._validation(state)
        if mean_score > self.goal_score:
            print("Mean score : ", mean_score)
        return mean_score > self.goal_score

    def step_cost(self, state, action, result=None):
        """The cost of taking this action from this state."""
        bag_id, gift_type  = action        
        return self.type_cost[gift_type]  # Override this if actions have different costs
                
    def _validation(self, state, count=5):
        scores = np.zeros(count)
        for c in range(count):
            score = 0
            for bag in state:
                total_weight_ = self.bag_weight_fn(bag)
                if total_weight_ < self.max_weight:
                    score += total_weight_
            scores[c] = score
        return np.mean(scores)

In [10]:
alpha = 0.85
goal_score = n_bags*max_weight*alpha
print("Goal score: ", goal_score)

Goal score:  2125.0


In [11]:
initial_state=tuple([()]*n_bags)
p = SantasBagsProblem(initial=initial_state,
                      gift_types=list(gift_types)[:n_types], 
                      available_gifts=available_gifts.copy(), 
                      max_weight=max_weight,    
                      type_cost=type_cost,
                      weight_fn=weight_fn,
                      bag_weight_fn=bag_weight,
                      goal_score=goal_score)

Define heuristic function :


In [12]:
gift_mean_weight = {}

for gift_type in gift_types:
    gift_mean_weight[gift_type] = weight_fn(gift_type, 1500)    

In [13]:
def h1(state):
    h = 0
    for bag in state:
        w = bag_weight(bag)
        h += max(max_weight*alpha/n_bags - w, 0.0)
    return h

def h2(state):     
    h = 0
    for bag in state:
        h += bag_weight(bag)
    h = max(goal_score - h, 0.0) / n_bags
    return h

def h12(state):     
    h1 = 0
    h2 = 0
    for bag in state:
        h1 += bag_weight(bag)
        w = bag_weight(bag)
        h2 += max(max_weight*alpha/n_bags - w, 0.0)
    h1 = max(goal_score - h1, 0.0) / n_bags    
    return max(h1, h2)


def h3(state):
    h = 0
    for bag in state:
        w = 0
        for gift in bag:
            gift_type = gift.split('_')[0]
            w += gift_mean_weight[gift_type]
        h += max(max_weight/n_bags - w, 0.0)
    return h

def h4(state):     
    h = 0
    for bag in state:
        w = 0
        for gift in bag:
            gift_type = gift.split('_')[0]
            w += gift_mean_weight[gift_type]
        h += w
    h = max(max_weight*n_bags - h, 0.0) / n_bags
    return h


def final_heuristic_fn(state):    
    return np.max(np.array([h12(state), h3(state), h4(state)]))

In [14]:
h12(initial_state), h3(initial_state), h4(initial_state)

(42.500000000000036, 50.0, 50.0)

In [15]:
from time import time

In [16]:
tic = time()
result = astar_search(p, final_heuristic_fn)
print(result)
print("Elapsed: ", time() - tic)

KeyboardInterrupt: 

In [ ]:
result.state

In [176]:
h12(result.state), h3(result.state), h4(result.state)

AttributeError: 'NoneType' object has no attribute 'state'

In [ ]:
p._validation(result.state)

In [20]:
state=[()]*10
ag=available_gifts.copy()
counter = 0
while len(state) < n_bags and counter < 10:
    counter += 1
    print("State len: ", len(state))
    p = SantasBagsProblem(initial=tuple(state),
                          gift_types=gift_types, 
                          available_gifts=ag,
                          max_weight=max_weight,    
                          type_cost=type_cost,
                          weight_fn=weight_fn,
                          bag_weight_fn=bag_weight,
                          goal_score=goal_score)
    tic = time()
    result = astar_search(p, final_heuristic_fn)
    if result is not None:
        print("- Got a result")
        state += result.state
    else:
        raise Exception('Result is none')
    print("- Elapsed: ", time() - tic)

State len:  10


Exception: Result is none

In [ ]:
state

In [ ]:
submission_file = '../results/submission_' + \
                  str(datetime.now().strftime("%Y-%m-%d-%H-%M")) + \
                  '.csv'

In [ ]:
def write_submission(state, filename):
    with open(filename, 'w') as w:
        w.write("Gifts\n")
        for bag in state:
            w.write(' '.join(bag) + '\n')
    
write_submission(result.state, submission_file)